#Задание 1. Написать теггер на данных с русским языком
#####- проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
#####- написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
#####- сравнить все реализованные методы, сделать выводы


In [83]:
import pyconll
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import tqdm
from tqdm import tqdm
import zipfile
import corus
from corus import load_ne5
import spacy
from spacy import displacy
import en_core_web_md
import requests
from bs4 import BeautifulSoup
import re


from nltk.tag import DefaultTagger
from sklearn.preprocessing import StandardScaler
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger
from nltk.corpus import names
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [8]:
!mkdir datasets

In [10]:
!wget -O /content/datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-Taiga/master/ru_taiga-ud-train.conllu
!wget -O /content/datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-Taiga/master/ru_taiga-ud-dev.conllu

--2022-10-25 11:03:15--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-Taiga/master/ru_taiga-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17229332 (16M) [text/plain]
Saving to: ‘/content/datasets/ru_syntagrus-ud-train.conllu’

/content/datasets/r 100%[===================>]  16.43M  --.-KB/s    in 0.1s    

2022-10-25 11:03:15 (135 MB/s) - ‘/content/datasets/ru_syntagrus-ud-train.conllu’ saved [17229332/17229332]

--2022-10-25 11:03:15--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-Taiga/master/ru_taiga-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubuserconten

In [45]:
full_train = pyconll.load_from_file('/content/datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('/content/datasets/ru_syntagrus-ud-dev.conllu')


In [46]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Снова ADV
приобрел VERB
дозу NOUN
, PUNCT

В ADP
женщине NOUN
важна ADJ
верность NOUN
, PUNCT
а CCONJ
не PART
красота NOUN
. PUNCT



In [47]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [48]:
fdata_train[666][:3]

[('Я', 'PRON'), ('—', 'PUNCT'), ('два', 'NUM')]

In [49]:
fdata_sent_test[666][:3]

['кречет', 'кречет', 'ратник']

In [50]:
default_tagger = nltk.DefaultTagger('NOUN')
display(default_tagger.tag(fdata_sent_test[666]), default_tagger.evaluate(fdata_test))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


[('кречет', 'NOUN'),
 ('кречет', 'NOUN'),
 ('ратник', 'NOUN'),
 ('воли', 'NOUN'),
 ('ты', 'NOUN'),
 ('летящий', 'NOUN'),
 ('сквозь', 'NOUN'),
 ('юдоли', 'NOUN'),
 ('раненый', 'NOUN'),
 ('пращею', 'NOUN'),
 ('туман', 'NOUN'),
 ('тимпан', 'NOUN'),
 ('веспасиан', 'NOUN')]

0.24167987321711568

In [51]:

unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(fdata_sent_test[666]), unigram_tagger.evaluate(fdata_test))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


[('кречет', None),
 ('кречет', None),
 ('ратник', None),
 ('воли', 'NOUN'),
 ('ты', 'PRON'),
 ('летящий', None),
 ('сквозь', 'ADP'),
 ('юдоли', None),
 ('раненый', None),
 ('пращею', None),
 ('туман', 'NOUN'),
 ('тимпан', None),
 ('веспасиан', None)]

0.6831418383518225

In [52]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.tag(fdata_sent_test[666]), bigram_tagger.evaluate(fdata_test))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


[('кречет', None),
 ('кречет', None),
 ('ратник', None),
 ('воли', 'NOUN'),
 ('ты', 'PRON'),
 ('летящий', None),
 ('сквозь', 'ADP'),
 ('юдоли', None),
 ('раненый', None),
 ('пращею', None),
 ('туман', 'NOUN'),
 ('тимпан', None),
 ('веспасиан', None)]

0.6859152139461173

In [53]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
display(trigram_tagger.tag(fdata_sent_test[666]), trigram_tagger.evaluate(fdata_test))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


[('кречет', None),
 ('кречет', None),
 ('ратник', None),
 ('воли', 'NOUN'),
 ('ты', 'PRON'),
 ('летящий', None),
 ('сквозь', 'ADP'),
 ('юдоли', None),
 ('раненый', None),
 ('пращею', None),
 ('туман', 'NOUN'),
 ('тимпан', None),
 ('веспасиан', None)]

0.6867076069730587

In [54]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NOUN') 
tag = backoff_tagger(fdata_train,  
                     [
                      UnigramTagger,
                      BigramTagger,
                      TrigramTagger
                     ],  
                     backoff = backoff) 

In [55]:
tag.evaluate(fdata_test) 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  """Entry point for launching an IPython kernel.


0.785756735340729

In [56]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [57]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM',
       'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X'],
      dtype='<U5')

In [58]:
i = 0
for text in test_tok:
    if text is None:
        test_tok[i] = 'None'
    i += 1

In [59]:
for vectorizer in tqdm([CountVectorizer, HashingVectorizer, TfidfVectorizer]):
    hvectorizer = vectorizer(ngram_range=(1, 5), analyzer='char')
    std_scaler = StandardScaler(with_mean=False)
    
    X_train = hvectorizer.fit_transform(train_tok)
    X_test = hvectorizer.transform(test_tok)

    X_train = std_scaler.fit_transform(X_train)
    X_test = std_scaler.fit_transform(X_test) 
    
    
    print('\n', X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter=100, n_jobs=5)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

  0%|          | 0/3 [00:00<?, ?it/s]


 (176631, 93146)


 33%|███▎      | 1/3 [01:03<02:07, 63.51s/it]

<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.8331022187004754

 (176631, 1048576)


 67%|██████▋   | 2/3 [07:13<04:03, 243.69s/it]

<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.8484548335974643

 (176631, 93146)


100%|██████████| 3/3 [08:33<00:00, 171.17s/it]

<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.8544968304278923


In [60]:
for vectorizer in tqdm([CountVectorizer, HashingVectorizer, TfidfVectorizer]):
    hvectorizer = vectorizer(ngram_range=(1, 5), analyzer='word')
    std_scaler = StandardScaler(with_mean=False)
    
    X_train = hvectorizer.fit_transform(train_tok)
    X_test = hvectorizer.transform(test_tok)

    X_train = std_scaler.fit_transform(X_train)
    X_test = std_scaler.fit_transform(X_test) 
    
    print('\n', X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter=100, n_jobs=5)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

  0%|          | 0/3 [00:00<?, ?it/s]


 (176631, 34451)


 33%|███▎      | 1/3 [00:33<01:06, 33.16s/it]

<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.49554278922345485

 (176631, 1048576)


 67%|██████▋   | 2/3 [06:37<03:48, 228.23s/it]

<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.5009904912836767

 (176631, 34451)


100%|██████████| 3/3 [07:28<00:00, 149.53s/it]

<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.4948494453248811


#Вывод
Векторное пространство показало лучшие метрики качества с опцией analyzer='char', accuracy_score = 0.85 и Тэггер = 0.68

 # Задание 2. Проверить, насколько хорошо работает NER
####a) проверить NER из nltk/spacy/deeppavlov
####b) написать свой NER, 
####- попробовать разные подходы
####- передаём в сетку токен и его соседей.
####- передаём в сетку только токен. свой вариант.
####c) сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


In [61]:
!wget  http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-10-25 12:31:24--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  1.91MB/s    in 1.0s    

2022-10-25 12:31:26 (1.91 MB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [64]:
arc = 'collection5.zip'
z = zipfile.ZipFile(arc)
directory_to_extract_to = ''
with zipfile.ZipFile(arc, 'r') as zip_file:
    zip_file.extractall(directory_to_extract_to)

In [65]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.4 MB/s 


In [68]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [70]:
path = 'Collection5/'
records = load_ne5(path)
doc = next(records).text

In [74]:
doc

'"Ювентус" обыграл "Милан" в Кубке Италии\r\n"Ювентус" обыграл "Милан" в матче 1/4 финала Кубка Италии. Как сообщает Reuters, встреча, состоявшаяся в среду, 9 января, в Турине, завершилась со счетом 2:1 в пользу хозяев поля - "Ювентуса".\r\n\r\nПервый гол был забит на шестой минуте матча нападающим "Милана" Стефаном Эль-Шаарави. На 12-й минуте "Ювентусу" удалось отыграться, автором гола стал Себастьян Джовинко.\r\n\r\nОсновное время матча завершилось со счетом 1:1. Победный мяч в ворота "Милана" отправил в дополнительное время Мирко Вучинич.\r\n\r\nТаким образом, "Ювентус" вышел в полуфинал Кубка Италии, где встретится с "Лацио". '

In [75]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }


{('Reuters', 'ORGANIZATION'), ('Италии', 'PERSON'), ('Таким', 'PERSON')}

In [76]:
doc = next(records).text

In [77]:
doc

'Пекин считает, что отношения с Вашингтоном "здоровые и стабильные"\r\n\r\n Китай называл правомерными действия властей Гонконга в отношении бывшего сотрудника американских спецслужб Эдварда Сноудена и считает, что отношения КНР с США продолжают здоровое и стабильное развитие, заявила в понедельник официальный представитель МИД КНР Хуа Чуньин на регулярном брифинге.\r\n\r\nСША требовали экстрадировать Сноудена, раскрывший информацию о прослушивании спецслужбами США телефонов граждан, но правительство Гонконга заявило, что у него не было никаких правовых оснований задерживать американца, поскольку запрос не полностью соответствовал местному законодательству. В воскресенье стало известно, что Сноуден прибыл из Гонконга в Москву и запросил убежища в Эквадоре. \r\n\r\n"Мы считаем, что китайско-американские отношения сохраняют здоровое и стабильное развитие, что отвечает интересам КНР и США и народов двух стран. Китайская сторона также готова прикладывать вместе с США усилия для неуклонного

In [78]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }


{('Reuters', 'ORGANIZATION'), ('Италии', 'PERSON'), ('Таким', 'PERSON')}

In [79]:
!python -m spacy info

2022-10-25 12:40:03.651345: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

============================== Info about spaCy ==============================

spaCy version    3.4.1                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.15                        
Pipelines        en_core_web_sm (3.4.1)        



In [81]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.0/en_core_web_md-2.2.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96.4 MB 100 kB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.0-py3-none-any.whl size=98072934 sha256=4e3a0ec00a8112c48ce4878d3b8170f3445c3d1390a65e8a4a5840fb76049b24
  Stored in directory: /root/.cache/pip/wheels/d1/e8/f9/fd0d324ec96ab55e0a8ae395183c22260a4eeaf166ade7ad22
Successfully built en-core-web-md


In [85]:
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

document = url_to_string('https://www.rt.com/russia/565149-end-of-russian-civil-war/')

nltk.pos_tag(nltk.word_tokenize(document))

[('The', 'DT'),
 ('Russian', 'NNP'),
 ('Civil', 'NNP'),
 ('War', 'NNP'),
 ('ended', 'VBD'),
 ('100', 'CD'),
 ('years', 'NNS'),
 ('ago', 'RB'),
 (':', ':'),
 ('Here', 'RB'),
 ("'s", 'VBZ'),
 ('how', 'WRB'),
 ('Western', 'JJ'),
 ('powers', 'NNS'),
 ('played', 'VBD'),
 ('a', 'DT'),
 ('significant', 'JJ'),
 ('part', 'NN'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('outcome', 'NN'),
 ('—', 'NNP'),
 ('RT', 'NNP'),
 ('Russia', 'NNP'),
 ('&', 'CC'),
 ('Former', 'NNP'),
 ('Soviet', 'NNP'),
 ('Union', 'NNP'),
 ('icon', 'NN'),
 ('bookmark-bicon', 'JJ'),
 ('bookmarkicon', 'NN'),
 ('cameraicon', 'NN'),
 ('checkicon', 'NN'),
 ('chevron', 'NN'),
 ('downicon', 'NN'),
 ('chevron', 'NN'),
 ('lefticon', 'NN'),
 ('chevron', 'NN'),
 ('righticon', 'NN'),
 ('chevron', 'NN'),
 ('upicon', 'JJ'),
 ('closeicon', 'NN'),
 ('v-compressicon', 'JJ'),
 ('downloadicon', 'NN'),
 ('editicon', 'NN'),
 ('v-expandicon', 'JJ'),
 ('fbicon', 'NN'),
 ('fileicon', 'NN'),
 ('filtericon', 'NN'),
 ('flag', 'NN'),
 ('ruicon', 'NN'),
 ('full'

In [87]:
from spacy.lang.ru import Russian
nlp = Russian()
doc = nlp("Зеленский попросил у Евросоюза $38 млрд на 2023 год")
print(doc)
for ent in doc:
    print(ent.text)

Зеленский попросил у Евросоюза $38 млрд на 2023 год
Зеленский
попросил
у
Евросоюза
$
38
млрд
на
2023
год


In [89]:
!python -m spacy download ru_core_news_lg

2022-10-25 12:48:36.149406: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 513.4 MB 14 kB/s 
     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 9.5 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=8af3e4b7725478d172ab7f89c70c385335ecc6bd28826303c6651ac098eee06d
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [90]:
import spacy
from spacy import displacy
import ru_core_news_lg

In [91]:
nlp = spacy.load('ru_core_news_lg')
doc = nlp("Зеленский попросил у Евросоюза $38 млрд на 2023 год")

print(doc, doc.ents)

for token in doc:
    print(token.text, token.pos_, token.dep_)  

displacy.render(doc, jupyter=True, style='dep')

Зеленский попросил у Евросоюза $38 млрд на 2023 год (Зеленский, Евросоюза)
Зеленский PROPN nsubj
попросил VERB ROOT
у ADP case
Евросоюза PROPN obl
$ SYM obj
38 NUM nummod
млрд NOUN obj
на ADP case
2023 ADJ amod
год NOUN obl


In [92]:
!python -m spacy download en_core_web_sm

2022-10-25 12:51:05.916669: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [96]:
txt = 'One hundred years ago, on October 25, 1922, the Russian Civil War drew to a close. It was on that day that the Provisional Priamurye Government in the Russian Far East, the last anti-Bolshevik Russian state enclave, ceased to exist.' + \
'The remnants of the White movement left Vladivostok. By that time, the territory of the former Russian Empire was almost entirely controlled by the Bolsheviks, although islands of resistance continued popping up sporadically in various parts of the country for several more years.'



In [97]:
txt

'One hundred years ago, on October 25, 1922, the Russian Civil War drew to a close. It was on that day that the Provisional Priamurye Government in the Russian Far East, the last anti-Bolshevik Russian state enclave, ceased to exist.The remnants of the White movement left Vladivostok. By that time, the territory of the former Russian Empire was almost entirely controlled by the Bolsheviks, although islands of resistance continued popping up sporadically in various parts of the country for several more years.'

In [100]:
import en_core_web_md

nlp = spacy.load('en_core_web_sm')

article = nlp(txt)
displacy.render(article, jupyter=True, style='ent')